In [12]:
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
train_df = pd.read_csv("dsjtzs_txfz_training.txt",header=None,delim_whitespace=True)
train_df.columns = ["id","points_str","target_point","label"]


train_df.set_index(["id"],inplace=True)

def f(s):
    s =str(s)
    return s[0:-1]
train_df["points_str"]= train_df["points_str"].apply(f)


def  point_sort(x,y):
    if (x[2]>y[2]):
        return 1
    elif (x[2]<y[2]):
        return -1
    else:
        return 0
def f(t):
    s=str(t["points_str"])
    splited=s.split(";")
    ret = []
    for item in splited:
        item_splited = item.split(",")
        item_tuple = ( int(item_splited[0]),  int(item_splited[1]) ,int(item_splited[2]) )
        ret.append(item_tuple)
    sorted_arr = sorted(ret,point_sort)
    return sorted_arr
train_df["points_sorted_list"]=train_df[["points_str"]].apply(f,axis=1)

def f(s):
    sorted_arr=s["points_sorted_list"]
    _sum_x =0 
    _sum_y=0 
    _count =0
    ret=[]
    for i in range(0,len(sorted_arr)-1):
        if (sorted_arr[i][2]!=sorted_arr[i+1][2]):
                if _count==0:
                    ret.append(sorted_arr[i])
                else:
                    _count +=1
                    _sum_x+=sorted_arr[i][0]
                    _sum_y+=sorted_arr[i][1]
                    average_x =float( _sum_x ) / _count
                    average_y =float(_sum_y)/_count
                    ret.append((average_x,average_y,sorted_arr[i][2]))
                    _count =0
                    _sum_x=0
                    _sum_y=0
        else: 
            _sum_x += sorted_arr[i][0]
            _sum_y+=sorted_arr[i][1]
            _count+=1
    
    index = len(sorted_arr)-1
    if (sorted_arr[index][2] == sorted_arr[index-1][2]):
        _count+=1
        _sum_x+=sorted_arr[index][0]
        _sum_y+=sorted_arr[index][1]
        average_x =float( _sum_x ) / _count
        average_y =float(_sum_y)/_count
        ret.append((average_x,average_y,sorted_arr[index][2]))
    else:
        ret.append(sorted_arr[index])
    return ret

train_df["points_diff_time_list"]=train_df[["points_sorted_list"]].apply(f,axis=1)

# print train_df.loc[1,"points_str"]
# print "-----"
# print train_df.loc[1,"points_sorted_list"]

# points 都有数据
#print pd.isnull(train_df["points"]).value_counts()

# 2600 个1 正例；400个0 负例 (id:2601-3000)
#train_df.label.value_counts()

# id 都是不同的
#train_df.id.value_counts().shape[0]

In [14]:
def f_x(s):
     
    ret =[]
    for item in s:
        ret.append(int(item[0]))
    return ret
        
train_df["points_x"]= train_df["points_diff_time_list"].apply(f_x)

def f_y (s):
     
    ret=[]
    for item in s:
        ret.append(int(item[1]))
    return ret
train_df["points_y"]=train_df["points_diff_time_list"].apply(f_y)

def f_time (s):
  
    ret=[]
    for item in s:
        ret.append(int(item[2]) )
    return ret
train_df["points_time"]=train_df["points_diff_time_list"].apply(f_time)

def f_target_point_x(s):
    s = str(s)
    splited = s.split(",")
    return float(splited[0])
train_df["target_point_x"]=train_df["target_point"].apply(f_target_point_x)

def f_target_point_y (s):
    s =str(s)
    splited =s.split(",")
    return float(splited[1])
train_df["target_point_y"]=train_df["target_point"].apply(f_target_point_y)



train_df =train_df[["points_x","points_y","points_time","target_point_x","target_point_y","target_point","label"]]

#去掉points就１个的, id有：493,2019　(points没有２个的，就不用删除２个的了)
def filter(s):
    if len(s)==1:
        return False
    return True
filters = train_df["points_x"].apply(filter)
train_df=train_df[filters]


print train_df.shape

(2998, 7)


In [ ]:
# plt.figure(figsize=(8,7),dpi=98)
# p1 = plt.subplot(211)
# p2 = plt.subplot(212)
# x = train_df.loc[1,"points_x"]
# y  = train_df.loc[1,"points_y"]
# target_x = train_df.loc[1,"target_point_x"]
# target_y = train_df.loc[1,"target_point_y"]
# print x
# print y
# p1.plot(x,y,'-',color='r',label="person")
# p1.plot(target_x,target_y,marker='*')
# p1.legend()
# plt.show()
ids = [23,24,2623,2634]
for id in ids:
    plt.figure(id)  #一个Figure对象可以包含多个子图（Axes）．指定了plt.figure(i)之后的操作就是在图表i 下进行的
    x = train_df.loc[id,"points_time"]
    y  = train_df.loc[id,"points_y"]
    target_x = train_df.loc[id,"target_point_x"]
    target_y = train_df.loc[id,"target_point_y"]
#     print x
#     print y
    print id
    print (target_x,target_y)
    print "-----------"
    if train_df.loc[id,"label"]==1:
        labelStr="person"
    else:
        labelStr="robort"
    plt.scatter(x,y,c="r",marker="*")
#  plt.plot(x,y,'-',color='r',label=labelStr)
 #  plt.plot(target_x,target_y,marker='*')
    plt.grid(True)
    plt.legend()
plt.show()

In [15]:
def  f (s):
    ret = []
    points_x= s["points_x"]
    points_time =s["points_time"]
    for x in range(1,len(points_x)):
        velocity = (float(points_x[x])-points_x[x-1])/(float(points_time[x])-points_time[x-1])
        ret.append(velocity)
    return ret
train_df["velocity_x"]= train_df[["points_x","points_time"]].apply(f,axis=1)


def f(s):
    ret =[]
    points_y = s["points_y"]
    points_time = s["points_time"]
    for x in range(1,len(points_y)):
        velocity = (float(points_y[x])-points_y[x-1])/(float(points_time[x])-points_time[x-1])
        ret.append(velocity)
    return ret
train_df["velocity_y"]=train_df[["points_y","points_time"]].apply(f,axis=1)

def f(s):
    ret =[]
    velocity_x =s["velocity_x"]
    times = s["points_time"]
    if len(velocity_x)>1:
        for x in range (1,len (velocity_x)):
            accelerate_x = (float(velocity_x[x])- velocity_x[x-1])/((float(times[x+1])+times[x])/2 - (float(times[x])+times[x-1])/2)
            ret.append(accelerate_x)
    return ret
train_df["accelerate_x"]=train_df[["velocity_x","points_time"]].apply(f,axis=1)


def f(s):
    ret = []
    velocity_y =s["velocity_y"]
    times=s["points_time"]
    if (len(velocity_y)>1):
        for x in range(1,len(velocity_y)):
            accelerate_y = (float(velocity_y[x])-velocity_y[x-1])/((float(times[x+1])+times[x])/2 - (float(times[x])+times[x-1])/2)
            ret.append(accelerate_y)
    return ret
train_df["accelerate_y"]=train_df[["velocity_y","points_time"]].apply(f,axis=1)




In [ ]:
import numpy as np
# # 对于point就一个点的，速度就是０
# train_df["velocity_x"]=train_df["velocity_x"].apply(lambda x: [0] if (len(x)==0) else x )
# train_df["velocity_y"]=train_df["velocity_y"].apply(lambda x: [0] if(len(x)==0) else x)

# #对于速度就一个值的，加速度为０
# train_df["accelerate_x"]=train_df["accelerate_x"].apply(lambda x: [0] if (len(x)==0) else x)
# train_df["accelerate_y"]=train_df["accelerate_y"].apply(lambda x: [0] if (len(x)==0) else x)

# 均值：速度　加速度
train_df["velocity_x_mean"]=train_df["velocity_x"].apply(lambda x: np.mean(np.array(x)))
train_df["velocity_y_mean"]=train_df["velocity_y"].apply(lambda x:np.mean(np.array(x)))
train_df["accelerate_x_mean"]=train_df["accelerate_x"].apply(lambda x:np.mean(np.array(x)))
train_df["accelerate_y_mean"]=train_df["accelerate_y"] .apply(lambda x:np.mean(np.array(x)))

# 方差：速度　加速度
train_df["velocity_x_var"]=train_df["velocity_x"].apply(lambda x:np.var(np.array(x)))
train_df["velocity_y_var"]=train_df["velocity_y"].apply(lambda x:np.var(np.array(x)))
train_df["accelerate_x_var"]=train_df["accelerate_x"].apply(lambda x:np.var(np.array(x)))
train_df["accelerate_y_var"]=train_df["accelerate_y"].apply(lambda x:np.var(np.array(x)))

# 最大值: 速度 加速度
train_df["max_velocity_x"] =train_df["velocity_x"].apply(lambda x: max(x))
train_df["max_velocity_y"] = train_df["velocity_y"].apply(lambda x:max(x))
train_df["max_accelerate_x"] = train_df["accelerate_x"].apply(lambda x:max(x))
train_df["max_accelerate_y"] = train_df["accelerate_y"].apply(lambda x:max(x))

# 最小值:速度 加速度
train_df["min_velocity_x"]= train_df["velocity_x"].apply(lambda x: min(x))
train_df["min_velocity_y"]=train_df["velocity_y"].apply(lambda x:min(x))
train_df["min_accelerate_x"]=train_df["accelerate_x"].apply(lambda x:min(x))
train_df["min_accelerate_y"]=train_df["accelerate_y"].apply(lambda x:min(x))

# count : 速度  
train_df["count_velocity"]= train_df["velocity_x"].apply(lambda x: len(x))
 


#25%: 速度　加速度
train_df["velocity_x_percentile_25"]=train_df["velocity_x"].apply(lambda x :np.percentile(x,25))
train_df["velocity_y_percentile_25"]=train_df["velocity_y"].apply(lambda x:np.percentile(x,25))
train_df["accelerate_x_percentile_25"]=train_df["accelerate_x"].apply(lambda x:np.percentile(x,25))
train_df["accelerate_y_percentile_25"]=train_df["accelerate_y"].apply(lambda x:np.percentile(x,25))

# 75% : 速度　加速度
train_df["velocity_x_percentile_75"]=train_df["velocity_x"].apply(lambda x:np.percentile(x,75))
train_df["velocity_y_percentile_75"]=train_df["velocity_y"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_x_percentile_75"]=train_df["accelerate_x"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_y_percentile_75"]=train_df["accelerate_y"].apply(lambda x:np.percentile(x,75))

# 50% :速度方差
train_df["velocity_x_percentile_50"]=train_df["velocity_x"].apply(lambda x:np.percentile(x,75))
train_df["velocity_y_percentile_50"]=train_df["velocity_y"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_x_percentile_50"]=train_df["accelerate_x"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_y_percentile_50"]=train_df["accelerate_y"].apply(lambda x:np.percentile(x,75))


train_df["x_mean"]=train_df["points_x"].apply(lambda x:np.mean(np.array(x)))
train_df["y_mean"]=train_df["points_y"].apply(lambda x:np.mean(np.array(x)))

# 路径中x 的平均值距离目标x的距离
def f(s):
    x_mean =s["x_mean"]
    target_point_x = s["target_point_x"]
    return x_mean-target_point_x
train_df["x_mean_dis_target_x"] = train_df[["x_mean","target_point_x"]].apply(f,axis=1)

# 路径中　y的平均值距离目标y的距离　
def f(s):
    y_mean =s["y_mean"]
    target_point_y = s["target_point_y"]
    return y_mean - target_point_y
train_df["y_mean_dis_target_y"]=train_df[["y_mean","target_point_y"]].apply(f,axis=1)

# 路径中每个点的x 距离目标x 的距离　（数组）
def f(s):
    points_x = s["points_x"]
    target_point_x =s["target_point_x"]
    ret=[]
    for i in len(points_x):
        ret.append(points_x[i]-target_point_x)
    return ret
train_df["x_dis_target_point_x"]=train_df[["points_x","target_point_x"]].apply(f,axis=1)

# 路径中点x 距离目标x 的最小距离，最大距离，等比分点，平均,方差
train_df["x_dis_target_point_x_min"]=train_df.x_dis_target_point_x.apply(lambda x:min(s))
train_df["x_dis_target_point_x_max"]=train_df.x_dis_target_point_x.apply(lambda x:max(s))
train_df["x_dis_target_point_x_25"]=train_df.x_dis_target_point_x.apply(lambda x: np.percentile(x,25))
train_df["x_dis_target_point_x_50"]=train_df.x_dis_target_point_x.apply(lambda x:np.percentile(x,50))
train_df["x_dis_target_point_x_75"]=train_df.x_dis_target_point_x.apply(lambda x:np.percentile(x,75))
train_df["x_dis_target_point_x_average"]=train_df.x_dis_target_point_x.apply(lambda x:np.mean(x))
train_df["x_dis_target_point_x_var"]=train_df.x_dis_target_point_x.apply(lambda x:np.var(x))

# 路径中每个点的y 距离目标y的 距离 （数组）
def f(s):
    points_y = s["points_y"]
    target_point_y = s["target_point_y"]
    ret = []
    for i in len(points_y):
        ret.append(points_y[i]-target_point_y)
    return ret
train_df["y_dis_target_point_y"]=train_df[["points_y","target_point_y"]].apply(f,axis=1)

# 路径中点y 距离目标y 的最小距离，最大距离，等比分点，平均,方差
train_df["y_dis_target_point_y_min"]=train_df.y_dis_target_point_y.apply(lambda x:min(s))
train_df["y_dis_target_point_y_max"]=train_df.y_dis_target_point_y.apply(lambda x:max(s))
train_df["y_dis_target_point_y_25"]=train_df.y_dis_target_point_y.apply(lambda x: np.percentile(x,25))
train_df["y_dis_target_point_y_50"]=train_df.y_dis_target_point_y.apply(lambda x:np.percentile(x,50))
train_df["y_dis_target_point_x_75"]=train_df.y_dis_target_point_y.apply(lambda x:np.percentile(x,75))
train_df["y_dis_target_point_y_average"]=train_df.y_dis_target_point_y.apply(lambda x:np.mean(x))
train_df["y_dis_target_point_y_var"]=train_df.y_dis_target_point_y.apply(lambda x:np.var(x))

# y 突变的次数

# x多少个不变
 
print train_df[:10]